In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/notebooks/")
from importlib import reload
import utils
reload(utils)
from utils import *

In [ ]:
comuni_piemonte = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
agricensus = pd.read_csv('data/raw/Unità agricole e superfici per zona altimentrica (IT1,DF_DCAT_CENSAGRIC2020_UA_CROPS_1,1.0).csv')
agricensus.info()

In [ ]:
agricensus.describe(include='all')

In [ ]:
# number of rows there should be
comuni_piemonte.shape[0]*len(agricensus['TYPE_OF_CROP'].unique())

In [ ]:
agricensus = agricensus.apply(lambda x: set_city_code(x, 'REF_AREA'), axis=1)
mask = agricensus['REF_AREA'].isin(comuni_piemonte['Codice Comune'])
agricensus[mask].info()

In [ ]:
# why that many?
agricensus_piemonte = agricensus[mask]
groups = agricensus_piemonte.groupby('REF_AREA')
count_istances = pd.Series([group.shape[0] for id, group in groups])
count_istances.value_counts()

In [ ]:
count_duplicates = pd.Series([group['TYPE_OF_CROP'].duplicated(keep='first').sum() for id, group in groups])
count_duplicates

What do they differ about?

In [ ]:
print(agricensus_piemonte['TIME_PERIOD'].value_counts())
print(agricensus_piemonte['DATAFLOW'].value_counts())
print(agricensus_piemonte['DATA_TYPE'].value_counts())
print(agricensus_piemonte['ALTIMETRIC_ZONE'].value_counts())

It is the altimetric zone and the data type(surface analysed). For each zone, I have all the type of crop. Let's verify it

In [ ]:
agricensus_piemonte['TYPE_OF_CROP'].unique()

In [ ]:
agricensus_piemonte['DATA_TYPE']

In [ ]:
actual_no = np.sum([len(group['DATA_TYPE'].unique())*len(group['TYPE_OF_CROP'].unique())*len(group['ALTIMETRIC_ZONE'].unique()) for id, group in groups])
print(actual_no)
print(agricensus_piemonte.shape[0])

Evidently the three variables are not independent (cannot combine woodlands with extensive agriculture)

In [ ]:
agricultural_land_usage_census = agricensus_piemonte.drop(agricensus.columns[8:], axis=1
                ).drop(['DATAFLOW', 'FREQ', 'TIME_PERIOD'], axis=1)
save_clean_data(agricultural_land_usage_census, 'agricultural_land_usage_census_2020')

---